In [1]:
import numpy as np
import numba as nb
import os
import cv2
import warnings
import pickle
import time
from math import floor
from skimage.measure import compare_psnr
from scipy.misc import imresize
from Functions import *
import matplotlib.image as mpimg
from multiprocessing import Pool

warnings.filterwarnings('ignore')

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:

testPath = 'set 14'
isKmeans = True

R = 3  # Upscaling factor=2 R = [ 2 3 4 ]
patchSize = 11  # Pacth Size=11
gradientSize = 9
Qangle = 24  # Quantization factor of angle =24
Qstrength = 3  # Quantization factor of strength =3
Qcoherence = 3  # Quantization factor of coherence =3

if isKmeans:
    # Kmeans-RAISR
    with open("Filters/Ckmeans_filter"+str(R), "rb") as fp:
        h = pickle.load(fp)

    with open("Filters/Ckmeans_Qfactor_str"+str(R), "rb") as sp:
        stre = pickle.load(sp)

    with open("Filters/Ckmeans_Qfactor_coh"+str(R), "rb") as cp:
        cohe = pickle.load(cp)
else:   
    # RAISR   
    with open("Filters/8x_filter"+str(R), "rb") as fp:
        h = pickle.load(fp)

    with open("Filters/8x_Qfactor_str"+str(R), "rb") as sp:
        stre = pickle.load(sp)

    with open("Filters/8x_Qfactor_coh"+str(R), "rb") as cp:
        cohe = pickle.load(cp)

kmeans = joblib.load('x'+str(R)+'_'+'1_3000_kmeans.model')
pca_matrix = io.loadmat('x'+str(R)+'_'+'1_3000_patch_PCA_matrix.mat')['p']
mean = io.loadmat('x'+str(R)+'_'+'1_3000_patch_mean_matrix.mat')['m']


filelist = make_dataset(testPath)

wGaussian = Gaussian2d([patchSize, patchSize], 2)
wGaussian = wGaussian/wGaussian.max()
wGaussian = np.diag(wGaussian.ravel())
imagecount = 1
patchMargin = floor(patchSize/2)
psnrRAISR = []
psnrBicubic = []
psnrBlending = []

In [3]:
def TestProcess(i):
    if (i < iteration - 1):
        offset_i = offset[i * batch:i * batch + batch]
        offset2_i = offset2[i * batch:i * batch + batch]
        grid = np.tile(gridon[..., None], [1, 1, batch]) + np.tile(offset_i, [patchSize, patchSize, 1])
    else:
        offset_i = offset[i * batch:im.size]
        offset2_i = offset2[i * batch:im.size]
        grid = np.tile(gridon[..., None], [1, 1, im.size - (iteration - 1) * batch]) + np.tile(offset_i,[patchSize, patchSize,1])
    f = im_LR.ravel()[grid]
    gx = im_GX.ravel()[grid]
    gy = im_GY.ravel()[grid]
    gx = gx.reshape((1, patchSize * patchSize, gx.shape[2]))
    gy = gy.reshape((1, patchSize * patchSize, gy.shape[2]))
    G = np.vstack((gx, gy))
    g1 = np.transpose(G, (2, 0, 1))
    g2 = np.transpose(G, (2, 1, 0))
    x = Gaussian_Mul(g1, g2, wGaussian)
    w, v = np.linalg.eig(x)
    idx = (-w).argsort()
    w = w[np.arange(np.shape(w)[0])[:, np.newaxis], idx]
    v = v[np.arange(np.shape(v)[0])[:, np.newaxis, np.newaxis], np.arange(np.shape(v)[1])[np.newaxis, :, np.newaxis], idx[:,np.newaxis,:]]
    thelta = np.arctan(v[:, 1, 0] / v[:, 0, 0])
    thelta[thelta < 0] = thelta[thelta < 0] + pi
    thelta = np.floor(thelta / (pi / Qangle))
    thelta[thelta > Qangle - 1] = Qangle - 1
    thelta[thelta < 0] = 0
    lamda = w[:, 0]
    u = (np.sqrt(w[:, 0]) - np.sqrt(w[:, 1])) / (np.sqrt(w[:, 0]) + np.sqrt(w[:, 1]) + 0.00000000000000001)
    lamda = np.searchsorted(stre, lamda)
    u = np.searchsorted(cohe, u)
    j = thelta * Qstrength * Qcoherence + lamda * Qcoherence + u
    j = j.astype('int')
    offset2_i = np.unravel_index(offset2_i, (H, W))
    
    if isKmeans:
        f_vec = np.reshape(f, (patchSize * patchSize, -1)) - mean
        f_vec_reduced = np.dot(pca_matrix, f_vec)
        t = kmeans.predict(f_vec_reduced.reshape(-1, 1))
        if i ==10:
            print(np.sum(t==0)/t.shape[0])
            print(np.sum(t==1)/t.shape[0])
            print(np.sum(t==2)/t.shape[0])
            print(np.sum(t==3)/t.shape[0])
    else:
        t = ((offset2_i[0] - patchMargin) % R) * R + ((offset2_i[1] - patchMargin) % R)
        if i ==10:
            print(np.sum(t==0)/t.shape[0])
            print(np.sum(t==1)/t.shape[0])
            print(np.sum(t==2)/t.shape[0])
            print(np.sum(t==3)/t.shape[0])
    
    filtertj = h[t, j]
    filtertj = filtertj[:, :, np.newaxis]
    patch = f.reshape((1, patchSize * patchSize, gx.shape[2]))
    patch = np.transpose(patch, (2, 0, 1))
    result = np.matmul(patch, filtertj)
    return result

In [4]:
print('Begin to process images:')
for image in filelist:
    print('\r', end='')
    print('' * 60, end='')
    print('\r Processing ' + str(imagecount) + '/' + str(len(filelist)) + ' image (' + image + ')')
    im_uint8 = cv2.imread(image)
    im_mp = mpimg.imread(image)
    if len(im_mp.shape) == 2:
        im_uint8 = im_uint8[:,:,0]
    im_uint8 = modcrop(im_uint8, R)
    if len(im_uint8.shape) > 2:
        im_ycbcr = BGR2YCbCr(im_uint8)
        im = im_ycbcr[:, :, 0]
    else:
        im = im_uint8
    im_double = im2double(im)
    H, W = im.shape
    region = (slice(patchMargin, H - patchMargin), slice(patchMargin, W - patchMargin))
    start = time.time()
    imL = imresize(im_double, 1 / R, interp='bicubic', mode='F')
    im_bicubic = imresize(imL, (H, W), interp='bicubic', mode='F')
    im_bicubic = im_bicubic.astype('float64')
    im_bicubic = np.clip(im_bicubic, 0, 1)
    im_LR = np.zeros((H+patchSize-1,W+patchSize-1))
    im_LR[(patchMargin):(H+patchMargin),(patchMargin):(W+patchMargin)] = im_bicubic
    im_result = np.zeros((H, W))
    im_GX, im_GY = np.gradient(im_LR)
    index = np.array(range(im_LR.size)).reshape(im_LR.shape)
    offset = np.array(index[0:H, 0:W].ravel())
    offset2 = np.array(range(im.size))
    gridon = index[0:patchSize, 0:patchSize]
    batch = 2000
    iteration = ceil(im.size / batch + 0.000000000000001)

    im_result = np.array([])

    i = range(iteration)
    p = Pool()
    im_in = p.map(TestProcess, i)

    for i in range(iteration):
        im_result = np.append(im_result, im_in[i])

    im_result = im_result.reshape(H, W)
    im_result = np.clip(im_result, 0, 1)

    end = time.time()
    print(end - start)

    im_blending = Blending2(im_bicubic, im_result)
    im_blending = Backprojection(imL, im_blending, 1) #Optional: Backprojection, which can slightly improve PSNR, especilly for large upscaling factor.
    im_blending = np.clip(im_blending, 0, 1)

    if len(im_uint8.shape) > 2:
        result_ycbcr = np.zeros((H, W, 3))
        result_ycbcr[:, :, 0] = im_blending * 255
        result_ycbcr[:, :, 1] = Prepare(im_ycbcr[:, :, 1], patchSize, R) * 255
        result_ycbcr[:, :, 2] = Prepare(im_ycbcr[:, :, 2], patchSize, R) * 255
        result_ycbcr = result_ycbcr[region].astype('uint8')
        result_RAISR = YCbCr2BGR(result_ycbcr)
    else:
        result_RAISR = im_result[region] * 255
        result_RAISR = result_RAISR.astype('uint8')

    im_result = im_result*255
    im_result = np.rint(im_result).astype('uint8')
    im_bicubic = im_bicubic*255
    im_bicubic = np.rint(im_bicubic).astype('uint8')
    im_blending = im_blending * 255
    im_blending = np.rint(im_blending).astype('uint8')

    PSNR_bicubic = compare_psnr(im[region], im_bicubic[region])
    PSNR_result = compare_psnr(im[region], im_result[region])
    PSNR_blending = compare_psnr(im[region], im_blending[region])
    PSNR_blending = max(PSNR_result, PSNR_blending)

    createFolder('./results/')
    cv2.imwrite('results/' + os.path.splitext(os.path.basename(image))[0] + '_result.bmp', result_RAISR)
    psnrRAISR.append(PSNR_result)
    psnrBicubic.append(PSNR_bicubic)
    psnrBlending.append(PSNR_blending)

    imagecount += 1


RAISR_psnrmean = np.array(psnrBlending).mean()
Bicubic_psnrmean = np.array(psnrBicubic).mean()


print('\r', end='')
print('' * 60, end='')
print('\r RAISR PSNR of '+ testPath +' is ' + str(RAISR_psnrmean))
print('\r Bicubic PSNR of '+ testPath +' is ' + str(Bicubic_psnrmean))

Begin to process images:
 Processing 1/14 image (set 14/foreman.bmp)
0.007
0.103
0.003
0.026
1.831017017364502
 Processing 2/14 image (set 14/man.bmp)
0.201
0.0
0.1285
0.1545
2.272819995880127
 Processing 3/14 image (set 14/pepper.bmp)
0.0795
0.199
0.0105
0.098
2.2575268745422363
 Processing 4/14 image (set 14/comic.bmp)
0.1235
0.1765
0.0075
0.144
1.6734352111816406
 Processing 5/14 image (set 14/coastguard.bmp)
0.002
0.0
0.396
0.0
1.744534969329834
 Processing 6/14 image (set 14/baboon.bmp)
0.262
0.0
0.178
0.07
2.1803929805755615
 Processing 7/14 image (set 14/monarch.bmp)
0.2015
0.0425
0.002
0.092
2.756720781326294
 Processing 8/14 image (set 14/flowers.bmp)
0.144
0.0305
0.0625
0.0625
1.9593002796173096
 Processing 9/14 image (set 14/face.bmp)
0.0
0.0
0.2825
0.0
1.7156181335449219
 Processing 10/14 image (set 14/ppt3.bmp)
0.002
0.442
0.0
0.006
2.7992820739746094
 Processing 11/14 image (set 14/zebra.bmp)
0.0045
0.8675
0.0
0.1205
2.5149271488189697
 Processing 12/14 image (set 14/brid

# Playground

In [5]:
kmeans.cluster_centers_

array([[-3.44721766],
       [ 2.60396842],
       [-1.12103449],
       [ 0.65496382]])

In [12]:
np.sum(kmeans.labels_==8) / kmeans.labels_.shape[0]

0.07130166666666667